## Twitter API scraping

In [1]:
import tweepy

API_KEY = "HZh8MFUydAVaIN50fYh2w5iVr"
API_SECRET = "BjkcY9H2tZhTHH2gDiyeByRJaBWExXxW523ZXvhy5jyh0mTlLn"
ACCESS_TOKEN = "301590896-xTlLyxVVYURtMRFDyhBa4BHEOEAHheCa9iEdicQv"
ACCESS_TOKEN_SECRET = "90soM5pLMXfuWkP1BIGVEDZFrBWkearvAiEeJ72JbW5rc"

auth = tweepy.OAuthHandler(API_KEY, API_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [2]:
def tweetSearch(query, limit = 100, language = "fr", date_since="2019-12-05", date_until="2019-12-06"):
    text = ""
    for tweet in tweepy.Cursor(api.search, q=query, lang=language, since=date_since, until=date_until).items(limit):
        text += tweet.text.lower() + ";;"
    return text

In [3]:
greve = tweetSearch("#Grève")

In [4]:
greve = greve.split(';;')
greve = set(greve)
greve = [i for i in greve if i] 

In [5]:
greve[:10]

[]

In [6]:
import pandas as pd
greve_twitter = pd.DataFrame(columns=['polarity','tweet'])

In [7]:
greve_twitter['tweet']=greve

## Preprocessing 

In [3]:
def clean_up(s):
    import re
    s= re.sub(r'http\S+|(www\.[^\s]+)|(@\S+)',' ',s)
    s=re.sub('[\d]',' ',s)
    s= re.sub('[^\w\s]',' ',s)
    s= re.sub('\s\s+',' ',s)
    s=s.lower().strip()
    return s
def stemmer(l):
    from nltk.stem.snowball import FrenchStemmer
    stemmer = FrenchStemmer()
    return list(map(lambda s: stemmer.stem(s), tweet))
def stop_word(l):
    from nltk.corpus import stopwords 
    stop_words = stopwords.words('french')
    stop_words.append('rt')
    return [x for x in l if x not in stop_words]
def tokenize(l):
    from nltk.tokenize import word_tokenize
    return word_tokenize(l)

## Cleaning final dataset 

In [21]:
greve_twitter['tweet'] = greve_twitter['tweet'].apply(lambda s: clean_up(s))

In [22]:
greve_twitter['tweet'] = greve_twitter['tweet'].apply(lambda s: tokenize(s))

In [23]:
greve_twitter['tweet'] = greve_twitter['tweet'].apply(lambda s: stem(s))

In [24]:
greve_twitter['tweet'] = greve_twitter['tweet'].apply(lambda s: remove_stopwords(s))

## Importing and cleaning train 

In [65]:
import pandas as pd
train= pd.read_csv('./nlp/tweets.csv', sep='^([^,]+),', engine='python', error_bad_lines=False, encoding='utf-8', index_col=[0])

In [66]:
train.head()

,polarity,statutnull
NaN,0,"- Awww, c'est un bummer. Tu devrais avoir davi..."
NaN,0,Est contrarié qu'il ne puisse pas mettre à jou...
NaN,0,J'ai plongé plusieurs fois pour la balle. A ré...
NaN,0,Tout mon corps a des démangeaisons et comme si...
NaN,0,"Non, il ne se comporte pas du tout. je suis en..."


In [67]:
train = train.reset_index(drop=True)

In [68]:
train.columns

Index(['polarity', 'statutnull'], dtype='object')

In [69]:
def clean_up(s):
    import re
    s= re.sub(r'http\S+|(www\.[^\s]+)|(@\S+)',' ',s)
    s=re.sub('[\d]',' ',s)
    s= re.sub('[^\w\s]',' ',s)
    s= re.sub('\s\s+',' ',s)
    s=s.lower().strip()
    return s
def tokenize(s):
    from nltk.tokenize import word_tokenize
    return word_tokenize(s)
def stem(tweet):
    from nltk.stem.snowball import FrenchStemmer
    stemmer = FrenchStemmer()
    return list(map(lambda s: stemmer.stem(s), tweet))
def remove_stopwords(l):
    from nltk.corpus import stopwords
    stop_words = stopwords.words('french')
    stop_words.append('rt')
    return [x for x in l if x not in stop_words]
train.statutnull=train.statutnull.astype(str)

In [70]:
train = train.sample(n=100000)

In [71]:
train.statutnull = train.statutnull.apply(lambda s: clean_up(s))

In [72]:
train.statutnull = train.statutnull.apply(lambda s: tokenize(s))

In [73]:
train.statutnull = train.statutnull.apply(lambda s: stem(s))

In [74]:
train.statutnull = train.statutnull.apply(lambda s: remove_stopwords(s))

In [75]:
train.rename(columns={'statutnull':'tweet'},inplace=True)

In [76]:
#train['tweet']=' '.join(train['tweet'])
train.head()

,polarity,tweet
450821,0,"[noueux, demain, semain, derni, écol, football..."
1399765,4,"[parc, ête, cher, humain]"
325960,0,"[auss, voitur, ressembl, zebr, reper, tout, ca..."
1376478,4,"[uh, oh, circulent, comm, vautour, temp, tir, ..."
255715,0,"[malheur, encor, plus, endorm, aim, être]"


In [27]:
train.to_csv('./nlp/train_tweets.csv')

In [28]:
train=pd.read_csv('./nlp/train_tweets.csv',index_col=[0])
train = train.reset_index(drop=True)

C:\Users\aline\AppData\Roaming\Python\Python37\site-packages\numpy\lib\arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [77]:
train.shape

(100000, 2)

In [78]:
train = train.dropna()

In [79]:
train.head()

,polarity,tweet
450821,0,"[noueux, demain, semain, derni, écol, football..."
1399765,4,"[parc, ête, cher, humain]"
325960,0,"[auss, voitur, ressembl, zebr, reper, tout, ca..."
1376478,4,"[uh, oh, circulent, comm, vautour, temp, tir, ..."
255715,0,"[malheur, encor, plus, endorm, aim, être]"


In [80]:
index_zero = train[(train['polarity'] != '0') & (train['polarity']!= '4')].index
train.drop(index_zero, inplace=True)

In [81]:
set(train['polarity'])

{'0', '4'}

## Building model 

In [82]:
all_words = []
import nltk
NUM_FEATURES = 5000
for index, value in train.tweet.iteritems():
    if value not in all_words:
        all_words += value

top_features = [x[0] for x in nltk.FreqDist(all_words).most_common(NUM_FEATURES)]

In [83]:
top_features[:10]

['a', 'dan', 'tout', 'plus', 'bon', 'aim', 'être', 'si', 'mainten', 'bien']

In [84]:
def build_features(words):
    features = {}
    for w in top_features:
        features[w] = (w in words)
    return features

In [85]:
featuresets = []

for index, row in train.iterrows():
    featuresets.append((build_features(row['tweet']), row['polarity']==4))

In [86]:
featuresets[:5]

[({'a': False,
   'dan': False,
   'tout': False,
   'plus': False,
   'bon': False,
   'aim': False,
   'être': False,
   'si': False,
   'mainten': False,
   'bien': False,
   'fair': False,
   'jour': False,
   'fait': False,
   'aujourd': False,
   'hui': False,
   'tel': False,
   'pens': False,
   'ça': False,
   'vrai': False,
   'lol': False,
   'peux': False,
   'quelqu': False,
   'regard': False,
   'merc': False,
   'vais': False,
   'travail': False,
   'manqu': False,
   'aller': False,
   'auss': False,
   'veux': False,
   'votr': False,
   'just': False,
   'temp': False,
   'comm': False,
   'avoir': False,
   'chos': False,
   'encor': False,
   'maison': False,
   'soir': False,
   'cet': False,
   'va': False,
   'heur': False,
   'journ': False,
   'peut': False,
   'oui': False,
   'nuit': False,
   'ser': False,
   'toujour': False,
   'oh': False,
   'pass': False,
   'voir': False,
   'esper': False,
   'amp': False,
   'tous': False,
   'nouveau': False,
   '

In [90]:
from nltk import NaiveBayesClassifier
from sklearn.model_selection import train_test_split
train1, test = train_test_split(featuresets, test_size=0.2)
classifier = NaiveBayesClassifier.train(train1)

In [91]:
classifier.show_most_informative_features()

Most Informative Features


In [92]:
import nltk
nltk.classify.accuracy(classifier, test)

1.0